In [1]:
import pandas as pd
import numpy as np
import requests
import csv
import json
from collections import OrderedDict
import scipy.stats

### Collect and filter dataset for useful values and assogn column names

In [2]:
language_data = pd.read_csv("DDW-C18-0000.csv")

In [3]:
language_data.head(7)

,"C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State,District,Area Name,Total/,Age-group,Number speaking second language,NaN,NaN,Number speaking third language,NaN,NaN
1,code,code,NaN,Rural/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Urban,NaN,Persons,Males,Females,Persons,Males,Females
3,1,2,3,4,5,6,7,8,9,10,11
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957


In [4]:
language_data = language_data.loc[5:,:]

In [5]:
['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3']

['State_Code',
 'District_Code',
 'Area_Name',
 'Total/Rural/Urban',
 'Age-group',
 'Number_speaking_second_language',
 'Males2',
 'Females2',
 'Number_speaking_third_language',
 'Males3',
 'Females3']

In [6]:
language_data_total = language_data.set_axis(['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3'], axis=1, inplace=False)
language_data_total

,State_Code,District_Code,Area_Name,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957
7,00,00,INDIA,Total,10-14,34488492,18133423,16355069,7254335,3831131,3423204
8,00,00,INDIA,Total,15-19,42424599,22750908,19673691,12626717,6792766,5833951
9,00,00,INDIA,Total,20-24,41344406,22386694,18957712,12834334,7067614,5766720
...,...,...,...,...,...,...,...,...,...,...,...
1080,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,25-29,13065,7012,6053,4364,2355,2009
1081,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,30-49,38406,21301,17105,13320,7824,5496
1082,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,50-69,12899,7903,4996,4177,2842,1335
1083,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,70+,1644,1016,628,432,305,127


In [7]:
indexNames = language_data_total[language_data_total['Area_Name'] == 'INDIA' ].index
language_data = language_data_total.drop(indexNames , inplace=False)

In [8]:
data = pd.read_csv("census_data.csv")
data

,State,District,Subdistt,Town/Village,Ward,EB,Level,Name,TRU,No_HH,...,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F
0,0,0,0,0,0,0,India,India,Total,249501663,...,5820403,1420348,491424,928924,5518450,3082613,2435837,728966109,291330383,437635726
1,0,0,0,0,0,0,India,India,Rural,168612897,...,5640822,1154975,386077,768898,3803400,1972348,1831052,485005760,200944045,284061715
2,0,0,0,0,0,0,India,India,Urban,80888766,...,179581,265373,105347,160026,1715050,1110265,604785,243960349,90386338,153574011
3,1,0,0,0,0,0,STATE,JAMMU & KASHMIR,Total,2119718,...,44570,26585,6780,19805,108457,56880,51577,8218589,3445572,4773017
4,1,0,0,0,0,0,STATE,JAMMU & KASHMIR,Rural,1553433,...,40936,21963,5757,16206,89232,45625,43607,5994979,2562471,3432508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,35,639,0,0,0,0,DISTRICT,North & Middle Andaman,Rural,25475,...,119,30,21,9,1047,654,393,65311,23633,41678
2024,35,639,0,0,0,0,DISTRICT,North & Middle Andaman,Urban,724,...,1,0,0,0,70,43,27,1707,616,1091
2025,35,640,0,0,0,0,DISTRICT,South Andaman,Total,59064,...,63,48,24,24,1311,933,378,141311,49720,91591
2026,35,640,0,0,0,0,DISTRICT,South Andaman,Rural,23767,...,62,42,20,22,777,550,227,57596,19971,37625


In [9]:
data = data[["Level","Name","TRU","TOT_P","TOT_M","TOT_F"]]

In [10]:
df = data[data['Level'] == "STATE"]

### Calculating Rural Data

In [11]:
#census_dataset = df[(df['TRU'] == "Rural") | (df['TRU'] == "Urban")]
census_dataset = df
census_dataset

,Level,Name,TRU,TOT_P,TOT_M,TOT_F
3,STATE,JAMMU & KASHMIR,Total,12541302,6640662,5900640
4,STATE,JAMMU & KASHMIR,Rural,9108060,4774477,4333583
5,STATE,JAMMU & KASHMIR,Urban,3433242,1866185,1567057
72,STATE,HIMACHAL PRADESH,Total,6864602,3481873,3382729
73,STATE,HIMACHAL PRADESH,Rural,6176050,3110345,3065705
...,...,...,...,...,...,...
2002,STATE,PUDUCHERRY,Rural,395200,194907,200293
2003,STATE,PUDUCHERRY,Urban,852753,417604,435149
2016,STATE,ANDAMAN & NICOBAR ISLANDS,Total,380581,202871,177710
2017,STATE,ANDAMAN & NICOBAR ISLANDS,Rural,237093,126287,110806


In [12]:
df1 = language_data.groupby(['Area_Name'])
df1.first()

,State_Code,District_Code,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
Area_Name,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,00,Total,Total,257425,142378,115047,72708,41846,30862
ANDHRA PRADESH,28,00,Total,Total,21583334,12193992,9389342,6344376,3836382,2507994
ARUNACHAL PRADESH,12,00,Total,Total,885945,478413,407532,418551,236666,181885
ASSAM,18,00,Total,Total,14460591,7979404,6481187,4352497,2636287,1716210
BIHAR,10,00,Total,Total,16761785,9443495,7318290,1682572,1028475,654097
CHANDIGARH,04,00,Total,Total,579920,322898,257022,321979,178557,143422
CHHATTISGARH,22,00,Total,Total,3384223,1820621,1563602,422569,245150,177419
DADRA & NAGAR HAVELI,26,00,Total,Total,198983,113211,85772,71516,44256,27260
DAMAN & DIU,25,00,Total,Total,116629,71260,45369,34602,21856,12746


In [13]:
group_state = list(df1.groups.keys())
group_state

['ANDAMAN & NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHANDIGARH',
 'CHHATTISGARH',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'JAMMU & KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'LAKSHADWEEP',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'NCT OF DELHI',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL']

In [14]:
State_dict = dict()

In [15]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z1 = z[z['Total/Rural/Urban'] == 'Rural']
    z1 = z1.iloc[0]
    
    z2 = z[z['Total/Rural/Urban'] == 'Urban']
    z2 = z2.iloc[0]

    rural_three_lang = int(z1['Number_speaking_third_language'])
    urban_three_lang = int(z2['Number_speaking_third_language'])
    
    State_Code = z1['State_Code']
    State_Name = key
    population_row = census_dataset[census_dataset["Name"] == State_Name]
    
    Total_row = population_row[population_row["TRU"] == "Total"]
    Rural_row = population_row[population_row["TRU"] == "Rural"]
    Urban_row = population_row[population_row["TRU"] == "Urban"]
    
    population = Total_row['TOT_P'].tolist()[0]

    rural_population = Rural_row['TOT_P'].tolist()[0]
    urban_population = Urban_row['TOT_P'].tolist()[0]
    
    #print("State_Name = ",State_Name)
    #print("State_Code = ",State_Code)
    #print("population = ",population)
    #print("rural_three_lang = ",rural_three_lang)
    #print("rural_population = ",rural_population)
    #print("urban_three_lang = ",urban_three_lang)
    #print("urban_population = ",urban_population)
    
    rural_prop = rural_three_lang/rural_population
    urban_prop = urban_three_lang/urban_population
    pool_prop = (rural_three_lang+urban_three_lang)/population           # Calculate Pool Proportion
    #print("rural_prop = ",rural_prop)
    #print("urban_prop = ",urban_prop)
    #print("pool_prop = ",pool_prop)
    
    rural_percent = (rural_three_lang*100)/rural_population
    urban_percent = (urban_three_lang*100)/urban_population
    SE = np.sqrt( ((pool_prop*(1-pool_prop))/rural_population) + ((pool_prop*(1-pool_prop))/urban_population) )   # Calculate Standard Error
    point_estimate = rural_prop-urban_prop           # Calculate Point Estimate
    #print("point_estimate = ",point_estimate)
    null_value = 0                                 # Null value for the Null Hypothesis
    z = (point_estimate - null_value)/SE           # Calculate z statistic
    p_value = scipy.stats.norm.sf(abs(z))*2        # Calculate p value for two tailed statistic
    #print("SE = ",SE)
    #print("z = ",z)
    
    
    #print("")
    #print("rural_percent = ",rural_percent)
    #print("urban_percent = ",urban_percent)
    #print("P value = ",p_value)
    State_dict[State_Code] = [round(urban_percent,3),round(rural_percent,3),p_value]
    
    #print("")
    #print("")

In [16]:
State_dict = OrderedDict(sorted(State_dict.items()))
State_dict

OrderedDict([('01', [24.394, 13.82, 0.0]),
             ('02', [9.653, 4.547, 0.0]),
             ('03', [34.609, 24.394, 0.0]),
             ('04', [30.944, 15.015, 0.0]),
             ('05', [3.844, 1.003, 0.0]),
             ('06', [6.872, 3.41, 0.0]),
             ('07', [8.264, 1.306, 0.0]),
             ('08', [2.33, 1.184, 0.0]),
             ('09', [2.962, 0.816, 0.0]),
             ('10', [3.571, 1.367, 0.0]),
             ('11', [41.169, 25.859, 0.0]),
             ('12', [39.713, 27.431, 0.0]),
             ('13', [40.509, 21.753, 0.0]),
             ('14', [28.86, 18.603, 0.0]),
             ('15', [6.095, 3.448, 0.0]),
             ('16', [9.06, 4.11, 0.0]),
             ('17', [25.464, 5.2, 0.0]),
             ('18', [31.445, 11.077, 0.0]),
             ('19', [7.62, 1.068, 0.0]),
             ('20', [8.33, 3.721, 0.0]),
             ('21', [30.901, 9.611, 0.0]),
             ('22', [3.792, 1.007, 0.0]),
             ('23', [3.464, 1.071, 0.0]),
             ('24', [21.51

In [17]:
a_file = open("geography-india-c.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state-code', 'urban-percentage', 'rural-percentage', 'p-value'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in State_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [18]:
data1 = pd.read_csv("geography-india-c.csv")
data1 = data1.to_string(index=False)
print("Displaying states and union territories of India data (Three or More Langage)")
print("")
print(data1)

Displaying states and union territories of India data (Three or More Langage)

 state-code  urban-percentage  rural-percentage      p-value
          1            24.394            13.820 0.000000e+00
          2             9.653             4.547 0.000000e+00
          3            34.609            24.394 0.000000e+00
          4            30.944            15.015 0.000000e+00
          5             3.844             1.003 0.000000e+00
          6             6.872             3.410 0.000000e+00
          7             8.264             1.306 0.000000e+00
          8             2.330             1.184 0.000000e+00
          9             2.962             0.816 0.000000e+00
         10             3.571             1.367 0.000000e+00
         11            41.169            25.859 0.000000e+00
         12            39.713            27.431 0.000000e+00
         13            40.509            21.753 0.000000e+00
         14            28.860            18.603 0.000000e+00
      

### Exacly 2 Language

In [19]:
State_dict = dict()

In [20]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z1 = z[z['Total/Rural/Urban'] == 'Rural']
    z1 = z1.iloc[0]
    
    z2 = z[z['Total/Rural/Urban'] == 'Urban']
    z2 = z2.iloc[0]

    rural_three_lang = int(z1['Number_speaking_third_language'])
    urban_three_lang = int(z2['Number_speaking_third_language'])
    
    rural_two_lang = int(z1['Number_speaking_second_language'])
    urban_two_lang = int(z2['Number_speaking_second_language'])
    
    rural_exactly_two = rural_two_lang - rural_three_lang
    urban_exactly_two = urban_two_lang - urban_three_lang
    
    State_Code = z1['State_Code']
    State_Name = key
    population_row = census_dataset[census_dataset["Name"] == State_Name]
    
    Total_row = population_row[population_row["TRU"] == "Total"]
    Rural_row = population_row[population_row["TRU"] == "Rural"]
    Urban_row = population_row[population_row["TRU"] == "Urban"]
    
    population = Total_row['TOT_P'].tolist()[0]

    rural_population = Rural_row['TOT_P'].tolist()[0]
    urban_population = Urban_row['TOT_P'].tolist()[0]
    
    #print("State_Name = ",State_Name)
    #print("State_Code = ",State_Code)
    #print("population = ",population)
    #print("rural_three_lang = ",rural_three_lang)
    #print("rural_population = ",rural_population)
    #print("urban_three_lang = ",urban_three_lang)
    #print("urban_population = ",urban_population)
    
    rural_prop = rural_exactly_two/rural_population
    urban_prop = urban_exactly_two/urban_population
    pool_prop = (rural_exactly_two+urban_exactly_two)/population
    #print("rural_prop = ",rural_prop)
    #print("urban_prop = ",urban_prop)
    #print("pool_prop = ",pool_prop)
    
    rural_percent = (rural_exactly_two*100)/rural_population
    urban_percent = (urban_exactly_two*100)/urban_population
    SE = np.sqrt( ((pool_prop*(1-pool_prop))/rural_population) + ((pool_prop*(1-pool_prop))/urban_population) )
    point_estimate = rural_prop-urban_prop
    #print("point_estimate = ",point_estimate)
    null_value = 0
    z = (point_estimate - null_value)/SE
    p_value = scipy.stats.norm.sf(abs(z))*2
    #print("SE = ",SE)
    #print("z = ",z)
    
    
    #print("")
    #print("rural_percent = ",rural_percent)
    #print("urban_percent = ",urban_percent)
    #print("P value = ",p_value)
    State_dict[State_Code] = [round(urban_percent,3),round(rural_percent,3),p_value]
    
    #print("")
    #print("")

In [21]:
State_dict = OrderedDict(sorted(State_dict.items()))
State_dict

OrderedDict([('01', [34.325, 31.933, 0.0]),
             ('02', [28.279, 11.345, 0.0]),
             ('03', [22.627, 16.445, 0.0]),
             ('04', [24.411, 25.422, 7.867753682658396e-05]),
             ('05', [26.409, 10.866, 0.0]),
             ('06', [25.585, 13.293, 0.0]),
             ('07', [32.78, 22.251, 0.0]),
             ('08', [16.594, 7.063, 0.0]),
             ('09', [23.908, 6.213, 0.0]),
             ('10', [20.511, 13.718, 0.0]),
             ('11', [33.561, 34.148, 2.7301400350198534e-05]),
             ('12', [31.901, 34.336, 4.61512525825916e-143]),
             ('13', [33.782, 35.477, 1.2149478321229065e-113]),
             ('14', [22.778, 28.778, 0.0]),
             ('15', [26.522, 20.001, 0.0]),
             ('16', [19.008, 32.711, 0.0]),
             ('17', [33.083, 14.776, 0.0]),
             ('18', [29.455, 32.874, 0.0]),
             ('19', [20.075, 10.139, 0.0]),
             ('20', [29.642, 24.171, 0.0]),
             ('21', [23.265, 19.074, 0.0]),
    

In [22]:
a_file = open("geography-india-b.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state-code', 'urban-percentage', 'rural-percentage', 'p-value'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in State_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [23]:
print("")

In [24]:
data1 = pd.read_csv("geography-india-b.csv")
data1 = data1.to_string(index=False)
print("Displaying states and union territories of India data (Exactly two Langage)")
print("")
print(data1)

Displaying states and union territories of India data (Exactly two Langage)

 state-code  urban-percentage  rural-percentage       p-value
          1            34.325            31.933  0.000000e+00
          2            28.279            11.345  0.000000e+00
          3            22.627            16.445  0.000000e+00
          4            24.411            25.422  7.867754e-05
          5            26.409            10.866  0.000000e+00
          6            25.585            13.293  0.000000e+00
          7            32.780            22.251  0.000000e+00
          8            16.594             7.063  0.000000e+00
          9            23.908             6.213  0.000000e+00
         10            20.511            13.718  0.000000e+00
         11            33.561            34.148  2.730140e-05
         12            31.901            34.336 4.615125e-143
         13            33.782            35.477 1.214948e-113
         14            22.778            28.778  0.0000

### Exactly 1 Language

In [25]:
State_dict = dict()

In [26]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z1 = z[z['Total/Rural/Urban'] == 'Rural']
    z1 = z1.iloc[0]
    
    z2 = z[z['Total/Rural/Urban'] == 'Urban']
    z2 = z2.iloc[0]
    
    rural_two_lang = int(z1['Number_speaking_second_language'])
    urban_two_lang = int(z2['Number_speaking_second_language'])
    
    State_Code = z1['State_Code']
    State_Name = key
    population_row = census_dataset[census_dataset["Name"] == State_Name]
    
    Total_row = population_row[population_row["TRU"] == "Total"]
    Rural_row = population_row[population_row["TRU"] == "Rural"]
    Urban_row = population_row[population_row["TRU"] == "Urban"]
    
    population = Total_row['TOT_P'].tolist()[0]

    rural_population = Rural_row['TOT_P'].tolist()[0]
    urban_population = Urban_row['TOT_P'].tolist()[0]
    
    rural_exactly_one = rural_population - rural_two_lang
    urban_exactly_one = urban_population - urban_two_lang
    
    #print("State_Name = ",State_Name)
    #print("State_Code = ",State_Code)
    #print("population = ",population)
    #print("rural_three_lang = ",rural_three_lang)
    #print("rural_population = ",rural_population)
    #print("urban_three_lang = ",urban_three_lang)
    #print("urban_population = ",urban_population)
    
    rural_prop = rural_exactly_one/rural_population
    urban_prop = urban_exactly_one/urban_population
    pool_prop = (rural_exactly_one+urban_exactly_one)/population
    #print("rural_prop = ",rural_prop)
    #print("urban_prop = ",urban_prop)
    #print("pool_prop = ",pool_prop)
    
    rural_percent = (rural_exactly_one*100)/rural_population
    urban_percent = (urban_exactly_one*100)/urban_population
    SE = np.sqrt( ((pool_prop*(1-pool_prop))/rural_population) + ((pool_prop*(1-pool_prop))/urban_population) )
    point_estimate = rural_prop-urban_prop
    #print("point_estimate = ",point_estimate)
    null_value = 0
    z = (point_estimate - null_value)/SE
    p_value = scipy.stats.norm.sf(abs(z))*2
    #print("SE = ",SE)
    #print("z = ",z)
    
    
    #print("")
    #print("rural_percent = ",rural_percent)
    #print("urban_percent = ",urban_percent)
    #print("P value = ",p_value)
    State_dict[State_Code] = [round(urban_percent,3),round(rural_percent,3),p_value]
    
    #print("")
    #print("")

In [27]:
State_dict = OrderedDict(sorted(State_dict.items()))
State_dict

OrderedDict([('01', [41.281, 54.247, 0.0]),
             ('02', [62.067, 84.108, 0.0]),
             ('03', [42.764, 59.161, 0.0]),
             ('04', [44.645, 59.563, 0.0]),
             ('05', [69.747, 88.131, 0.0]),
             ('06', [67.544, 83.297, 0.0]),
             ('07', [58.956, 76.443, 0.0]),
             ('08', [81.076, 91.753, 0.0]),
             ('09', [73.13, 92.971, 0.0]),
             ('10', [75.918, 84.914, 0.0]),
             ('11', [25.27, 39.993, 0.0]),
             ('12', [28.386, 38.232, 0.0]),
             ('13', [25.709, 42.769, 0.0]),
             ('14', [48.362, 52.619, 0.0]),
             ('15', [67.383, 76.551, 0.0]),
             ('16', [71.932, 63.18, 0.0]),
             ('17', [41.452, 80.025, 0.0]),
             ('18', [39.1, 56.049, 0.0]),
             ('19', [72.305, 88.793, 0.0]),
             ('20', [62.028, 72.108, 0.0]),
             ('21', [45.834, 71.315, 0.0]),
             ('22', [78.283, 89.317, 0.0]),
             ('23', [77.681, 89.86, 0

In [28]:
a_file = open("geography-india-a.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state-code', 'urban-percentage', 'rural-percentage', 'p-value'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in State_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [29]:
print("")

In [30]:
data1 = pd.read_csv("geography-india-a.csv")
data1 = data1.to_string(index=False)
print("Displaying states and union territories of India data (Exactly one Langage)")
print("")
print(data1)

Displaying states and union territories of India data (Exactly one Langage)

 state-code  urban-percentage  rural-percentage       p-value
          1            41.281            54.247  0.000000e+00
          2            62.067            84.108  0.000000e+00
          3            42.764            59.161  0.000000e+00
          4            44.645            59.563  0.000000e+00
          5            69.747            88.131  0.000000e+00
          6            67.544            83.297  0.000000e+00
          7            58.956            76.443  0.000000e+00
          8            81.076            91.753  0.000000e+00
          9            73.130            92.971  0.000000e+00
         10            75.918            84.914  0.000000e+00
         11            25.270            39.993  0.000000e+00
         12            28.386            38.232  0.000000e+00
         13            25.709            42.769  0.000000e+00
         14            48.362            52.619  0.0000